The data files train.csv and test.csv contain gray-scale images of hand-drawn digits, from zero through nine.

Each image is 28 pixels in height and 28 pixels in width, for a total of 784 pixels in total. Each pixel has a single pixel-value associated with it, indicating the lightness or darkness of that pixel, with higher numbers meaning darker. This pixel-value is an integer between 0 and 255, inclusive.

The training data set, (train.csv), has 785 columns. The first column, called "label", is the digit that was drawn by the user. The rest of the columns contain the pixel-values of the associated image.

Each pixel column in the training set has a name like pixelx, where x is an integer between 0 and 783, inclusive. To locate this pixel on the image, suppose that we have decomposed x as x = i * 28 + j, where i and j are integers between 0 and 27, inclusive. Then pixelx is located on row i and column j of a 28 x 28 matrix, (indexing by zero).

For example, pixel31 indicates the pixel that is in the fourth column from the left, and the second row from the top, as in the ascii-diagram below.

Visually, if we omit the "pixel" prefix, the pixels make up the image like this:

000 001 002 003 ... 026 027
028 029 030 031 ... 054 055
056 057 058 059 ... 082 083
 |   |   |   |  ...  |   |
728 729 730 731 ... 754 755
756 757 758 759 ... 782 783 
The test data set, (test.csv), is the same as the training set, except that it does not contain the "label" column.

Your submission file should be in the following format: For each of the 28000 images in the test set, output a single line containing the ImageId and the digit you predict. For example, if you predict that the first image is of a 3, the second image is of a 7, and the third image is of a 8, then your submission file would look like:

ImageId,Label
1,3
2,7
3,8 
(27997 more lines)
The evaluation metric for this contest is the categorization accuracy, or the proportion of test images that are correctly classified. For example, a categorization accuracy of 0.97 indicates that you have correctly classified all but 3% of the images.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib.pylab import plt

In [ ]:
train = pd.read_csv('./data/train.csv', delimiter=',', header=0)
test = pd.read_csv('./data/test.csv', delimiter=',', header=0)

In [ ]:
# for index, row in train.iterrows():
#    key = row['label']
#    print('Label: ',str(key))
#    data = np.array(row[1:]).reshape((28, 28))
#    plt.imshow(data, cmap=plt.get_cmap('gray'))
#    plt.show()

In [ ]:
# train['img'] = train.apply(lambda row: np.array(row[1:]).reshape((28, 28)), axis=1)

# Unfortunately it's not possible to store a 2d array in a Serie of pandas dataframe,
# I would require to use a Panel (i.e. Dataframe of Dataframe, I don't see how to do that with Scikit easily)
# So hopefully neural network model can take a row as input and interpret that as a 28x28 image array

Open questions:
- How to not trip the ML classifier when the digit is in a corner and not in the center ?
- What if colors are inverted ?

Ideas :
- Generate more data by translating data to the left/right/up/down
- Sharpen the image and add to the data with opencv/imagemagick
- scale up or down the data

In [ ]:
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import GridSearchCV

from sklearn.cross_validation import cross_val_score

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier

import time

In [ ]:
# Keras neural Network

# Get dimensions
nb_samples = train.shape[1] - 1 # 784 features
nb_classes = pd.get_dummies(train['label']).shape[1] #10 possible values for label

# Function to create model, required for KerasClassifier
# using a MultiLayerPerceptron
def create_model():
    # create model
    model = Sequential([
        Dense(128, input_dim=nb_samples),
        Activation('relu'),
        Dropout(0.15),
        Dense(128),
        Activation('relu'),
        Dropout(0.25),
        Dense(nb_classes),
        Activation('softmax')
    ])
        
    # Compile model
    model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
    return model

clf_keras = KerasClassifier(build_fn=create_model, nb_epoch=100, batch_size=512, verbose=1)

# fix random seed for reproducibility
seed = 777
np.random.seed(seed)

In [ ]:
featurelist = (''.join(['pixel',str(i)]) for i in range(784))
#featurelist generator

In [ ]:
mapper = DataFrameMapper(
    [ ([pixel], StandardScaler()) for pixel in featurelist]
)
# Do we need StandardScaler() ? It's the same scale of color for all pixels

In [ ]:
pipe = Pipeline([
    ("featurize", mapper),
    ("mlp", clf_keras)
])

In [ ]:
########################Helper functions ################
##### Cross Validation
def crossval():
    # split = ShuffleSplit(n_splits=10) #compat issue with sklearn pandas "object is not iterable"
    cv = cross_val_score(pipe, X_train, y_train, cv=10, n_jobs=1) #Note : don't use parallel crossval  and GPU computation
    print("Cross Validation Scores are: ", cv.round(3))
    print("Mean CrossVal score is: ", round(cv.mean(),3))
    print("Std Dev CrossVal score is: ", round(cv.std(),3))
    return cv
def output():
    predictions = pipe.predict(test)
    result = pd.DataFrame(predictions)
    result.index+=1
    result.index.names=['ImageId']
    result.columns=['Label']
    result.to_csv(time.strftime("%Y-%m-%d_%H%M-") + 'keras.csv', header=True)

In [ ]:
X_train = train
y_train = train['label']

In [ ]:
temp_cv = crossval()

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
output()